# Tweet classification with naive bayes

For this notebook we are going to implement a naive bayes classifier for classifying tweets about Trump or Obama based on the words in the tweet. Recall that for two events A and B the bayes theorem says

$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)} $$

where P(A) and P(B) is the ***class probabilities*** and P(B|A) is called ***conditional probabilities***. this gives us the probability of A happening, given that B has occurred. So as an example if we want to find the probability of "is this a tweet about Trump given that it contains the word "president" " we will obtain the following 

$$ P(\text{"Trump"}|\text{"president" in tweet}) = \frac{P("\text{"president" in tweet}|\text{"Trump"})P(\text{"Trump"})}{P("\text{"president" in tweet})} $$

This means that to find the probability of "is this a tweet about Trump given that it contains the word "president" " we need the probability of "president" being in a tweet about Trump, the probability of a tweet being about Trump and the probability of "president" being in a tweet. 

Similarly if we want to obtain the opposite "is this a tweet about Obama given that it contains the word "president" "
we get 

$$ P(\text{"Obama"}|\text{"president" in tweet}) = \frac{P(\text{"president" in tweet}|\text{"Obama"})P(\text{"Obama"})}{P(\text{"president" in tweet})} $$

where we need the probability of "president" being in a tweet about Obama, the probability of a tweet being about Obama and the probability of "president" being in a tweet. 

We can now build a classifier where we compare those two probabilities and whichever is the larger one it's classified as 

if P("Trump"|"president" in tweet) $>$ P("Obama"|"president" in tweet)
    
   Tweet is about Trump

else
   
   Tweet is about Obama

Now let's expand this to handle multiple features and put the Naive assumption into bayes theroem. This means that if features are independent we have 

$$ P(A,B) = P(A)P(B) $$

This gives us:

$$ P(A|b_1,b_2,...,b_n) = \frac{P(b_1|A)P(b_2|A)...P(b_n|A)P(A)}{P(b_1)P(b_2)...P(b_n)} $$

or

$$ P(A|b_1,b_2,...,b_n) = \frac{\prod_i^nP(b_i|A)P(A)}{P(b_1)P(b_2)...P(b_n)} $$


So with our previous example expanded with more words "is this a tweet about Trump given that it contains the word "president" and "America" " gives us 

$$ P(\text{"Trump"}|\text{"president", "America" in tweet}) = \frac{P(\text{"president" in tweet}|\text{"Trump"})P(\text{"America" in tweet}|\text{"Trump"})P(\text{"Trump"})}{P(\text{"president" in tweet})P(\text{"America" in tweet})} $$

As you can see the denominator remains constant which means we can remove it and the final classifier end up

$$y = argmax_A P(A)\prod_i^nP(b_i|A) $$

In [3]:
#stuff to import
import pandas as pd
import numpy as np
import random
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
assert pd.__version__ == "1.2.1", "Looks like you don't have the same version of pandas as us!"
assert np.__version__ == "1.19.4", "Looks like you don't have the same version of numpy as us!"
assert sklearn.__version__ == "0.24.0", "Looks like you don't have the same version of sklearn as us!"

AssertionError: Looks like you don't have the same version of numpy as us!

Load the data and explore

In [4]:
df_t = pd.read_csv('trump_20200530.csv')
trump_tweets = df_t['text']

df_t = pd.read_csv('Tweets-BarackObama.csv')
obama_tweets = df_t['Tweet-text']

tweet_data = trump_tweets.append(obama_tweets, ignore_index=True)
tweet_labels = np.array(['T' for _ in range(len(trump_tweets))] + ['O' for _ in range(len(obama_tweets))])


In [5]:
lab, counts = np.unique(tweet_labels, return_counts=True)
print('Number of tweets about ', lab[0], ': ', counts[0])
print('Number of tweets about ', lab[1], ': ', counts[1])

Number of tweets about  O :  6851
Number of tweets about  T :  18467


As you can see we have many more Trump than Obama Tweets so simlpy guessing that a tweet is a Trump tweet already gives you a classifier that is correct about 70% of the time, but we can do better than this.

Now lets split the data into a training set and a test set using scikit-learns train_test_split function 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [6]:
#Split data into train_tweets, test_tweets, train_labels and test_labels

from sklearn.model_selection import train_test_split

train_tweets, test_tweets, train_labels, test_labels = train_test_split(
     tweet_data, tweet_labels
)


What we need to build our classifier is "probability of tweet about Obama" P(O) , "probability of tweet about Trump" P(T), "probability of word in tweet given tweet about Obama" P(w|O) and "probability of word in tweet given tweet about Trump" P(w|T). Start by calculating the probability that a tweet is about Obama and trump respectively 

In [17]:
o_label = [label for label in train_labels if label=='O']

P_O = len(o_label)/len(tweet_labels) 
P_T = 1 - P_O


0.20222766411248913 0.7977723358875108 1.0


For P(w|O), P(w|T) we need to count how many tweets each word occur in. Count the number of tweets each word occurs in and store in the word counter. An entry in the word counter is for instance  {'president': 'O':87, 'T': 100} meaning president occurs in 87 words about Obaman and 100 tweets about Trump. Be aware that we are not interested in calculating multiple occurances of the same word in the same tweet.  For each word convert it to lower case. You can use Python's [lower](https://www.w3schools.com/python/ref_string_lower.asp). Another handy Python string method is [split](https://www.w3schools.com/python/ref_string_split.asp).

In [25]:
word_counter = {}
for (tweet, label) in zip(train_tweets, train_labels):
# ... Count number of tweets each word occurs in and store in word_counter where an entry looks like ex. {'word': 'O':98, 'T':10}
    words = tweet.lower().split()
    for word in words:              
        if word in word_counter:  
            word_counter[word][label] = word_counter[word][label] + 1
        else: 
            word_counter[word] = {}
            if label == 'O':
                word_counter[word]['O'] = 1
                word_counter[word]['T'] = 0
            else:
                word_counter[word]['T'] = 1
                word_counter[word]['O'] = 0
    

Lets work with a smaller subset of words. Find the 100 most occuring words in tweet data.

In [26]:
nr_of_words_to_use = 100
popular_words = sorted(word_counter.items(), key=lambda x: x[1]['O'] + x[1]['T'], reverse=True)
popular_words = [x[0] for x in popular_words[:nr_of_words_to_use]]

print(popular_words)

['the', 'to', 'and', 'of', 'a', 'in', 'is', 'for', 'rt', 'on', 'that', 'our', 'are', 'i', 'with', 'will', 'be', 'we', 'you', 'this', 'have', 'president', 'great', 'it', 'they', '&amp;', 'at', 'obama', 'was', 'has', 'not', 'by', 'all', 'he', 'from', 'as', 'people', 'who', 'just', 'no', 'your', 'very', 'my', '—president', 'so', 'their', 'more', 'about', 'if', 'his', 'do', 'democrats', 'thank', 'but', 'new', 'an', 'get', 'now', 'been', 'up', 'what', 'than', 'trump', '-', 'out', 'time', 'american', 'or', 'should', 'big', 'news', 'fake', 'many', 'would', 'can', 'there', 'want', 'make', 'one', 'never', 'country', 'when', 'today', 'u.s.', 'america', '@realdonaldtrump', 'house', 'good', 'congress', 'me', 'like', 'going', 'how', '@realdonaldtrump:', 'much', 'united', 'even', 'over', 'must', 'only']


Now lets compute P(w|O), P(w|T) for the popular words

In [27]:
P_w_given_t = {}
P_w_given_o = {}
for w in popular_words:
    word = word_counter[w]
    obama = word['O']
    trump = word['T']
    
    tot = obama + trump
    
    perc_obama = obama/tot
    perc_trump = trump/tot
    
    P_w_given_o[w] = perc_obama
    P_w_given_t[w] = perc_trump
    

In [28]:
classifier = {
    'basis'  : popular_words,
    'P(T)'   : P_O,
    'P(O)'   : P_T,
    'P(w|O)' : P_w_given_o,
    'P(w|T)' : P_w_given_t
    }   

Write a tweet_classifier function that takes your trained classifier and a tweet and returns wether it's about Trump or Obama unsing the popular words selected. Note that if there are words in the basis words in our classifier that are not in the tweet we have the opposite probabilities i.e P(w_1 occurs )*  P(w_2 does not occur) * .... if w_1 occurs and w_2 does not occur. The function should return wether the tweet is about Obama or Trump i.e 'T' or 'O'

In [29]:
def tweet_classifier(tweet, classifier_dict):
    """ param tweet: string containing tweet message
        param classifier: dict containing 'basis' - training words
                                          'P(T)' - class probabilities
                                          'P(O)' - class probabilities
                                          'P(w|O)' - conditional probabilities
                                          'P(w|T)' - conditional probabilities
        
        return: either 'T' or 'O'
    """
    words_in_tweet = np.unique([x.lower() for x in tweet.split()])
    
    # ... Code for classifying tweets using the naive bayes classifier
    prob_trump = 1
    prob_obama = 1
    for word in words_in_tweet:
        if word in P_w_given_t: 
            t = P_w_given_t[word]
            prob_trump = prob_trump*t
        
        if word in P_w_given_o:
            o = P_w_given_o[word]
            prob_obama = prob_obama*o

    if prob_obama*P_O > prob_trump*P_T:
        return 'O'
    else:
        return 'T'
    
        
        

In [30]:
def test_classifier(classifier, test_tweets, test_labels):
    total = len(test_tweets)
    correct = 0
    for (tweet,label) in zip(test_tweets, test_labels):
        predicted = tweet_classifier(tweet,classifier)
        if predicted == label:
            correct = correct + 1
    return(correct/total)

In [31]:
acc = test_classifier(classifier, test_tweets, test_labels)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.7834
